# NE-model hypertuning

This notebook provides a way to test hypertuning of NE-model data across all its weather years.

>**NOTE!**
>You need to have `tsam_df_dict.pkl` available for this script to run.
>This file is produced in `representative-period-processing.ipynb` after formatting raw data for TSAM.

This script consists of the following main steps:
1. Python setup and loading preprocessed input data.
2. TSAM aggregation based on user settings.
3. TSAM hypertuning using the aggregation from the previous step as the basis.
4. Hypertuning diagnostics.

## 1. Setup and input data

Imports the necessary packages,
suppresses TSAM warnings to avoid excessive printing,
and loads the preprocessed data from `representative-period-processing.ipynb`.

In [ ]:
## Import necessary packages

import pickle # Load preprocessed input data to save time.
import os # Check if results exist.
import tsam.timeseriesaggregation as tsam # Timeseries aggregation.
from multiprocessing import cpu_count # Count CPUs
import pickle # Save hypertuning results to save time.
# Suppress warnings from TSAM
import warnings

warnings.filterwarnings("ignore")

In [ ]:
## Load input data

with open("tsam_df_dict.pkl", "rb") as file:
    tsam_df_dict = pickle.load(file)

## 2. TSAM aggregation

Provides settings for TSAM aggregation to play with,
and then proceeds to do the aggregation.

>**NOTE!**
>The TSAM aggregation settings are used as the basis for the hypertuning.
>Furthermore, the given `noTypicalPeriods * hoursPerPeriod / <full_year_hours>` is used to calculate
>the desired `reduction_factor` for the hypertuning.

In [ ]:
## Configure TSAM aggregation
# Copied from `representative-period-processing.ipynb`
# (see https://tsam.readthedocs.io/en/latest/timeseriesaggregationDoc.html)

# Main settings you might be interested in tweaking.
noTypicalPeriods = 4 # Number of representative periods. (Periods for hypertuning)
hoursPerPeriod = 168 # Hours per representative period. (Segments for hypertuning?)
extremePeriodMethod = "replace_cluster_center" # Method to integrate extreme periods?
clusterMethod = "hierarchical" # Select clustering method. (`hierarchical` or `k_medoids` recommended)

# Calculate a hash based on the main settings to archive results.
settings_hash = f"{noTypicalPeriods}x{hoursPerPeriod}h-{extremePeriodMethod}-{clusterMethod}"

# Auxiliary settings you shoudldn't need to touch.
resolution = 1 # Resolution of input data in hours, shouldn't need to be touched.
rescaleClusterPeriods = False # Don't rescale periods, we don't use that data anyhow.
segmentation = True # Segmentation to enable hypertuning, doesn't impact the clustering results.
numericalTolerance = 1e-6 # Set numerical tolerance for TSAM so it stops complaining (doesn't seem to work for some reason).

In [ ]:
## TSAM time series aggregation
# Copied from `representative-period-processing.ipynb`

tsam_dict = dict() # Initialize dict to store TSAM aggregation per year.
for (year, data) in tsam_df_dict.items():
    aggregation = tsam.TimeSeriesAggregation( ## Define TSAM aggregation.
        data,
        noTypicalPeriods=noTypicalPeriods,
        hoursPerPeriod=hoursPerPeriod,
        clusterMethod=clusterMethod,
        resolution=resolution,
        rescaleClusterPeriods=rescaleClusterPeriods,
        extremePeriodMethod=extremePeriodMethod,
        segmentation=segmentation,
        numericalTolerance=numericalTolerance,
    )
    aggregation.createTypicalPeriods() ## Run TSAM aggregation.
    tsam_dict[year] = aggregation

## 3. TSAM hypertuning

Parallel TSAM hypertuning over the full year set.
In order to save time, hypertuning results are saved after their `settings_hash`,
calculated based on the main settings for the TSAM aggregation in the previous section.
The user can force redo hypertuning using the settings below,
in addition to manually tweaking the `num_workers` and `reduction_factor`.

>**NOTE!**
>The hypertuning will likely take tens of minutes of not hours to run over the entire dataset!
>Caution is advised.

In [ ]:
## Hypertuning settings

# Option to force hypertune.
force_hypertune = False
hypertune = (force_hypertune or not os.path.exists(f"{settings_hash}.pkl"))

# Set number of processors for multiprocessing. (CPU count - 1 default)
num_workers = cpu_count() - 1

# Data reduction factor calculated based on given aggregation settings.
reduction_factor = (
    noTypicalPeriods * hoursPerPeriod / len(next(iter(tsam_df_dict.values())))
)
(hypertune, num_workers, reduction_factor)

In [ ]:
## Hypertuning across all years.
# NOTE! This is likely going to take tens of minutes!
# Seems to be around ~15-20 min, but this might depend on aggregation settings?
# For some reason, Jupyter notebooks don't work with multiprocessing directly.

if hypertune:
    from multihyper import parallel_hypertuning
    hypertuned = parallel_hypertuning(tsam_dict, reduction_factor, num_workers)
    with open(f"{settings_hash}.pkl", "wb") as file: # Save results for future use.
        pickle.dump(hypertuned, file)
else:
    with open(f"{settings_hash}.pkl", "rb") as file: # Load previous results.
        hypertuned = pickle.load(file)

## 4. Hypertuning diagnostics

Examine the results of hypertuning.
We're mainly interested in how TSAM thinks each year should be represented,
and if this changes from year to year.

In [ ]:
## Calculate original TSAM aggregation accuracy indicators for comparison
# Parallelized to save time, otherwise takes surprisingly long.

from multihyper import parallel_diagnostics
tsam_diag = parallel_diagnostics(tsam_dict, num_workers)

# Overall, it seems that TSAM generally does better with a lot of short periods,
# instead of a limited number of longer periods.
# At least with all the NE-model data formatted as is.

In [ ]:
## Examine results, first for the desired aggregation.

tsam_diag["1990"]

In [ ]:
## Then for the hypertuned one

hypertuned["1990"]

In [ ]:
## Check counts for different aggregations
# This actually requires hypertuning to have been run.

segments_and_periods = [ # Extract segment and period information from hypertuning results.
    (seg, per) for (year, (seg, per, diag)) in hypertuned.items()
]
hypertuned_counts = { # Count unique segment-period pairs.
    seg_per: segments_and_periods.count(seg_per)
    for seg_per in set(segments_and_periods)
}
hypertuned_counts # Reported in hours x periods

# Updated to contain all timeseries.
# Seems like with too many timeseries, hypertuning is more or less useless?
# Or maybe it has something to do with the extreme period method?
# 4x24h -> 96x1h?
# 8x24h -> 192x1h? 
# 12x24h -> 288x1h?
# 4x168h -> 51x13h
# 5x168h -> Crashes due to insufficient memory...
# 8x168h -> Crashes due to insufficient memory...